In [2]:
import sys
sys.path.append('../')
from collections import defaultdict
import pandas as pd
import numpy as np
from scipy.stats import mannwhitneyu
from roverdata.db import DataWarehouse
from taxes_increase import utils

## Get LTV Data

In [3]:
LTV = pd.read_csv('../LTV.csv')
LTV180 = LTV.loc[LTV.dates == 180]
new = LTV180.loc[LTV180.new_repeat == 'new']
new_account = LTV180.loc[LTV180.new_account == 'new']

## Get Sample Size Data

In [3]:
samples = pd.read_csv('../retrans.csv')
samples_new = samples.loc[samples['new_repeat']=='new'][['service','num_owners']].groupby(['service']).sum()
samples_new_account = samples.loc[samples['new_account']=='new'][['service','num_owners']].set_index('service')

In [4]:
def project_30_day_samples(df, samples_col):
    """from df with column `service` and inputted samples column (containing 30 day samples)
    , project out 45 and 60 days"""
    df = df[samples_col].reset_index().rename(columns={samples_col:'30_day'})
    df['45_day'] = (df['30_day']*1.5).astype(int)
    df['60_day'] = (df['30_day']*2).astype(int)
    return df.set_index('service')


expected_xx_day_samples_new = project_30_day_samples(df = samples_new, samples_col = 'num_owners')
expected_xx_day_samples_new_account = project_30_day_samples(df = samples_new_account, samples_col = 'num_owners')

## Estimate Distribution of Sample Statistic

#### Estimate mean of means and standard error of means of sample statistic through simulation
http://blog.analytics-toolkit.com/2017/statistical-significance-non-binomial-metrics-revenue-time-site-pages-session-aov-rpu/

In [5]:
def sample_data(df, metric, samples, simulations):
    """
    given a dataframe `df` with a metric of interst column `metric`,
    select `samples` number of samples,
    repeat for `simulations` number of simulations
    
    Returns:
        meanOfMeans (float)
        seOfMeans (float)
    """
    metric_series = df[metric]
    means = np.array([metric_series.sample(n=samples, replace=True).mean() for i in range(simulations)])
    return means.mean(), means.std()

In [6]:
def run_sample_data(df, expected_xx_day_samples):
    """loop through the services, metrics, and sample sizes and estimate mean of means and std of means"""
    dist_stats_dict = {}
    
    for service in df.service.unique():
        df_service = df.loc[df['service'] == service]
        samples = expected_xx_day_samples.loc[service].to_dict()
        for days, samples in samples.items():
            for metric in ['gmv', 'nrt', 'stays']:
                mom, seom = sample_data(df_service, metric, samples, 1000)
                dist_stats_dict[(service, metric, days)] = (samples, mom, seom)  
                
    return pd.DataFrame.from_dict(dist_stats_dict, orient='index', columns=['samples','meanOfMeans','SEOfMeans'])

In [7]:
dist_stats_new = run_sample_data(
    df=new, 
    expected_xx_day_samples = expected_xx_day_samples_new
)
dist_stats_new_account = run_sample_data(
    df=new_account, 
    expected_xx_day_samples = expected_xx_day_samples_new_account
)

## Calculate minimum effect size for 95% power at 95% significance

In [8]:
from statsmodels.stats.power import tt_ind_solve_power

In [9]:
def dict_to_df(d, col_name):
    """convert dictionary with key = (service, metric, sample_days) to a pandas dataframe 
    with columns = col_name"""
    df = pd.DataFrame.from_dict(d, orient='index').rename(columns={0:col_name})
    midx = pd.MultiIndex.from_tuples(df.index.values.tolist(), names =('service', 'metric', 'samples')) 
    df = df.set_index(midx)
    df = df.pivot_table(index = ['service','samples'],  columns = ['metric'], values = col_name)
    return df

In [45]:
def calculate_mde(dist_stats):
    """Given the mean of means and standard deviation of means,
    Calculated the difference in means that would yeild 95% power at 95% significance
    """
    drop_dict = {}
    value = None

    for row in dist_stats.iterrows():
        index = row[0]
        data = row[1]
        nobs = int(data.samples*(2/3)*(1/2))
        mean = data.meanOfMeans
        se = data.SEOfMeans
        value = tt_ind_solve_power(effect_size=None, 
                                   nobs1=nobs, 
                                   alpha=.05, 
                                   power=.95, 
                                   ratio=1.0, 
                                   alternative='larger')
        drop_pct = 1 - (mean-(value*(se*(nobs**0.5))))/mean 
        drop_dict[index] = drop_pct
        
    drop_df = dict_to_df(drop_dict, 'drop')
    return drop_df*100


### MDE: New Customers

In [46]:
calculate_mde(dist_stats_new)

metric                     gmv        nrt      stays
service     samples                                 
*           30_day    5.243153   5.298154   5.470972
            45_day    4.501466   4.376314   4.439713
            60_day    3.705967   3.750442   4.077652
dog-walking 30_day   13.828163  13.540090  12.396662
            45_day   11.464241  11.502015  10.088191
            60_day   10.115175   9.998297   8.666581
drop-in     30_day   12.095776  12.217807  11.850778
            45_day    9.862144  10.046307  10.211611
            60_day    8.399081   8.684361   8.373505
overnight   30_day    6.156505   6.264898   6.249366
            45_day    5.281501   5.170180   5.122543
            60_day    4.486558   4.391708   4.399682

### MDE: New Accounts

In [29]:
calculate_mde(dist_stats_new_account)

metric                     gmv        nrt      stays
service     samples                                 
*           30_day    7.039586   6.839958   7.195719
            45_day    5.689205   5.803814   5.775493
            60_day    5.017123   4.925299   5.311064
dog-walking 30_day   17.937864  17.398971  15.453574
            45_day   15.006511  14.660083  12.952098
            60_day   13.081287  12.827295  11.155992
drop-in     30_day   15.344262  15.437471  15.291979
            45_day   12.524476  12.384092  12.245775
            60_day   10.507581  11.006046  10.810222
overnight   30_day    8.477425   8.192489   8.376794
            45_day    6.693400   6.614949   7.164113
            60_day    6.168300   6.057379   6.114535

## Calculate power for 6% drop at 95% significance

In [34]:
def calculate_power(dist_stats):
    """Given the mean of means and standard deviation of means,
    Calculated the power given a minimum effect size of 6% and 95% significance
    """

    power_dict = {}
    value = None

    for row in dist_stats.iterrows():
        index = row[0]
        data = row[1]
        nobs = int(data.samples*(2/3)*(1/2))
        mean = data.meanOfMeans
        se = data.SEOfMeans
        drop_pct = .06
        standardized_effect = (mean-mean*(1-drop_pct))/(se*(nobs**0.5))
        value = tt_ind_solve_power(effect_size=standardized_effect, 
                                   nobs1=nobs, 
                                   alpha=.05, 
                                   power=None, 
                                   ratio=1.0, 
                                   alternative='larger')
        power_dict[index] = value
    
    power_df = dict_to_df(power_dict, 'power')
    return power_df*100


### Power: New Customers

In [35]:
calculate_power(dist_stats_new)

metric                     gmv        nrt      stays
service     samples                                 
*           30_day   98.297948  98.126042  97.516698
            45_day   99.692649  99.791643  99.745135
            60_day   99.999995  99.985137  99.930181
dog-walking 30_day   41.393020  42.579992  47.901851
            45_day   53.064734  52.839773  62.238382
            60_day   62.038995  62.904161  73.652515
drop-in     30_day   49.480926  48.830853  50.826722
            45_day   63.930701  62.548452  61.336951
            60_day   75.965530  73.499654  76.188280
overnight   30_day   94.076402  93.393418  93.493374
            45_day   98.179928  98.510547  98.639132
            60_day   99.706322  99.781252  99.775583

### Power: New Accounts

In [36]:
calculate_power(dist_stats_new_account)

metric                     gmv        nrt      stays
service     samples                                 
*           30_day   87.678161  89.267514  86.394322
            45_day   96.596627  96.045996  96.186294
            60_day   98.896950  99.092790  98.084768
dog-walking 30_day   29.305357  30.488403  35.658848
            45_day   37.087976  38.268290  45.188845
            60_day   44.592566  45.775762  54.951722
drop-in     30_day   35.998753  35.708592  36.163420
            45_day   47.254521  47.965943  48.683500
            60_day   59.237765  55.906029  57.184556
overnight   30_day   75.283745  77.769284  76.158677
            45_day   90.389408  90.972068  86.656664
            60_day   94.003888  94.670335  94.331629